In [1]:
import numpy as np;
import pickle
import pandas as pd 

In [ ]:
# LOAD
# SPLIT 
# TRAIN AND TEST

In [2]:
data = pickle.load(open('./data_pickle_emotion.pickle', mode='rb'))

In [3]:
x = np.array(data['data'])   # independent var
y = np.array(data['label'])  # dependent var

In [4]:
x.shape, y.shape 

((34, 1, 128), (34,))

In [5]:
x = x.reshape(-1, 128)
x.shape

(34, 128)

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, train_size=0.8, random_state = 0)

In [8]:
xtrain.shape, xtest.shape, ytrain.shape, ytest.shape

((27, 128), (7, 128), (27,), (7,))

In [ ]:
# train ml

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [ ]:
# Logistic Regression

In [10]:
model_logistic = LogisticRegression()
model_logistic.fit(xtrain, ytrain)

LogisticRegression()

In [11]:
def get_report(model, xtrain, xtest, ytrain, ytest):
    y_pred_train = model.predict(xtrain)
    y_pred_test = model.predict(xtest)

    # accuracy 
    acc_train = accuracy_score(ytrain, y_pred_train)
    acc_test = accuracy_score(ytest, y_pred_test)

    # f1 score 
    f1_score_train = f1_score(ytrain, y_pred_train, average='macro')
    f1_score_test = f1_score(ytest, y_pred_test, average='macro')

    print("Accuracy train: ", acc_train)
    print("Accuracy test: ", acc_test)
    print("F1 score train: ", f1_score_train)
    print("F1 score test: ", f1_score_test)

In [12]:
get_report(model_logistic,  xtrain, xtest, ytrain, ytest)

Accuracy train:  0.3333333333333333
Accuracy test:  0.2857142857142857
F1 score train:  0.1425
F1 score test:  0.125


In [17]:
# Support Vector Machines

In [13]:
model_svc = SVC(probability = True)
model_svc.fit(xtrain, ytrain)

SVC(probability=True)

In [14]:
get_report(model_svc,  xtrain, xtest, ytrain, ytest)

Accuracy train:  0.37037037037037035
Accuracy test:  0.2857142857142857
F1 score train:  0.1686868686868687
F1 score test:  0.11111111111111112


In [ ]:
# Random forest

In [15]:
model_rf = RandomForestClassifier(n_estimators=10)
model_rf.fit(xtrain, ytrain)

RandomForestClassifier(n_estimators=10)

In [16]:
get_report(model_rf,  xtrain, xtest, ytrain, ytest)

Accuracy train:  1.0
Accuracy test:  0.5714285714285714
F1 score train:  1.0
F1 score test:  0.5166666666666666


In [ ]:
# Voting classifier 

In [17]:
model_voting = VotingClassifier(estimators=[
    ('logistic', LogisticRegression()),
    ('svm', SVC(probability=True)),
    ('rf', RandomForestClassifier())
], voting='soft', weights=[2, 3, 1])

In [18]:
model_voting.fit(xtrain, ytrain)

VotingClassifier(estimators=[('logistic', LogisticRegression()),
                             ('svm', SVC(probability=True)),
                             ('rf', RandomForestClassifier())],
                 voting='soft', weights=[2, 3, 1])

In [19]:
get_report(model_voting, xtrain, xtest, ytrain, ytest)

Accuracy train:  0.7407407407407407
Accuracy test:  0.5714285714285714
F1 score train:  0.43530701754385964
F1 score test:  0.3125


In [ ]:
# Parameter tuning

In [20]:
from sklearn.model_selection import GridSearchCV

In [21]:
model_grid = GridSearchCV(model_voting,
                         param_grid={
                             'svm__C' : [3, 5, 7, 10],
                             'svm__gamma' : [0.1, 0.3, 0.5],
                             'rf__n_estimators' : [5, 10, 20],
                             'rf__max_depth' : [3, 5, 7],
                             'voting' : ['soft', 'hard']
                         }, scoring='accuracy', cv=3, n_jobs=1, verbose=2)

In [22]:
model_grid.fit(xtrain, ytrain)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.0s


C:\Users\HP\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"


[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.5, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.5, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.5, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.5, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.5, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gam

[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=5, svm__gamma=0.1, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=5, svm__gamma=0.1, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=5, svm__gamma=0.3, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=5, svm__gamma=0.3, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=5, svm__gamma=0.3, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=5, svm__gamma=0.3, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=5, svm__gamma=0.3, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=5, svm__gamma=0.3, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=5, svm__gamma=0.5, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=5

[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=5, svm__gamma=0.3, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=5, svm__gamma=0.3, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=5, svm__gamma=0.5, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=5, svm__gamma=0.5, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=5, svm__gamma=0.5, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=5, svm__gamma=0.5, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=5, svm__gamma=0.5, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=5, svm__gamma=0.5, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=7, svm__gamma=0.1, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=7

...

[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=7, svm__gamma=0.5, voting=hard; total time=   0.0s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=7, svm__gamma=0.5, voting=hard; total time=   0.0s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=7, svm__gamma=0.5, voting=hard; total time=   0.0s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=10, svm__gamma=0.1, voting=soft; total time=   0.0s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=10, svm__gamma=0.1, voting=soft; total time=   0.0s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=10, svm__gamma=0.1, voting=soft; total time=   0.0s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=10, svm__gamma=0.1, voting=hard; total time=   0.0s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=10, svm__gamma=0.1, voting=hard; total time=   0.0s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=10, svm__gamma=0.1, voting=hard; total time=   0.0s
[CV] END rf__max_depth=5, rf__n_estimators=10, sv

[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=10, svm__gamma=0.3, voting=soft; total time=   0.0s
[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=10, svm__gamma=0.3, voting=soft; total time=   0.0s
[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=10, svm__gamma=0.3, voting=soft; total time=   0.0s
[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=10, svm__gamma=0.3, voting=hard; total time=   0.0s
[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=10, svm__gamma=0.3, voting=hard; total time=   0.0s
[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=10, svm__gamma=0.3, voting=hard; total time=   0.0s
[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=10, svm__gamma=0.5, voting=soft; total time=   0.0s
[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=10, svm__gamma=0.5, voting=soft; total time=   0.0s
[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=10, svm__gamma=0.5, voting=soft; total time=   0.0s
[CV] END rf__max_depth=5, rf__n_estimators=20,

[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=3, svm__gamma=0.1, voting=soft; total time=   0.0s
[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=3, svm__gamma=0.1, voting=soft; total time=   0.0s
[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=3, svm__gamma=0.1, voting=soft; total time=   0.0s
[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=3, svm__gamma=0.1, voting=hard; total time=   0.0s
[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=3, svm__gamma=0.1, voting=hard; total time=   0.0s
[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=3, svm__gamma=0.1, voting=hard; total time=   0.0s
[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.0s
[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.0s
[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.0s
[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=3

[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.0s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.0s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=3, svm__gamma=0.3, voting=hard; total time=   0.0s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=3, svm__gamma=0.3, voting=hard; total time=   0.0s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=3, svm__gamma=0.3, voting=hard; total time=   0.0s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=3, svm__gamma=0.5, voting=soft; total time=   0.0s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=3, svm__gamma=0.5, voting=soft; total time=   0.0s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=3, svm__gamma=0.5, voting=soft; total time=   0.0s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=3, svm__gamma=0.5, voting=hard; total time=   0.0s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=3

GridSearchCV(cv=3,
             estimator=VotingClassifier(estimators=[('logistic',
                                                     LogisticRegression()),
                                                    ('svm',
                                                     SVC(probability=True)),
                                                    ('rf',
                                                     RandomForestClassifier())],
                                        voting='soft', weights=[2, 3, 1]),
             n_jobs=1,
             param_grid={'rf__max_depth': [3, 5, 7],
                         'rf__n_estimators': [5, 10, 20],
                         'svm__C': [3, 5, 7, 10], 'svm__gamma': [0.1, 0.3, 0.5],
                         'voting': ['soft', 'hard']},
             scoring='accuracy', verbose=2)

In [23]:
model_best_estimator = model_grid.best_estimator_

In [24]:
model_grid.best_score_

0.4444444444444444

In [ ]:
# Save pickle

In [25]:
pickle.dump(model_best_estimator, open('../models/ml_face_emotion.pkl', mode='wb'))